## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ponta Do Sol,PT,32.6667,-17.1000,71.01,72,24,2.15,few clouds
1,1,Chuy,UY,-33.6971,-53.4616,41.76,82,24,4.85,few clouds
2,2,Radhan,PK,27.1833,67.9500,91.29,62,1,8.37,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,37.06,87,75,8.05,light rain
4,4,Norman Wells,CA,65.2820,-126.8329,64.44,68,75,25.32,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Rikitea,PF,-23.1203,-134.9692,75.00,75,10,14.61,clear sky
10,10,Vila Velha,BR,-20.3297,-40.2925,75.15,82,0,11.50,clear sky
11,11,Avera,US,33.1940,-82.5271,83.71,71,100,4.83,overcast clouds
17,17,Kapaa,US,22.0752,-159.3190,84.52,66,40,13.80,scattered clouds
25,25,Hilo,US,19.7297,-155.0900,87.28,74,0,10.36,clear sky
28,28,Akyab,MM,20.1500,92.9000,80.53,86,99,16.13,overcast clouds
33,33,Puerto Colombia,CO,10.9878,-74.9547,84.29,85,69,18.54,broken clouds
34,34,Albany,US,42.6001,-73.9662,81.41,67,21,1.01,few clouds
36,36,Vaini,TO,-21.2000,-175.2000,80.76,74,20,11.50,few clouds
42,42,Bubaque,GW,11.2833,-15.8333,80.89,80,100,9.78,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             247
City                   247
Country                247
Lat                    247
Lng                    247
Max Temp               247
Humidity               247
Cloudiness             247
Wind Speed             247
Current Description    247
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Rikitea,PF,75.00,clear sky,-23.1203,-134.9692,
10,Vila Velha,BR,75.15,clear sky,-20.3297,-40.2925,
11,Avera,US,83.71,overcast clouds,33.1940,-82.5271,
17,Kapaa,US,84.52,scattered clouds,22.0752,-159.3190,
25,Hilo,US,87.28,clear sky,19.7297,-155.0900,
28,Akyab,MM,80.53,overcast clouds,20.1500,92.9000,
33,Puerto Colombia,CO,84.29,broken clouds,10.9878,-74.9547,
34,Albany,US,81.41,few clouds,42.6001,-73.9662,
36,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,
42,Bubaque,GW,80.89,overcast clouds,11.2833,-15.8333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = np.nan
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Rikitea,PF,75.00,clear sky,-23.1203,-134.9692,People ThankYou
10,Vila Velha,BR,75.15,clear sky,-20.3297,-40.2925,Hotel Vitória Palace
11,Avera,US,83.71,overcast clouds,33.1940,-82.5271,NaN
17,Kapaa,US,84.52,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
25,Hilo,US,87.28,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
28,Akyab,MM,80.53,overcast clouds,20.1500,92.9000,Hotel Memory
33,Puerto Colombia,CO,84.29,broken clouds,10.9878,-74.9547,Hotel El Emigrante
34,Albany,US,81.41,few clouds,42.6001,-73.9662,NaN
36,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,Keleti Beach Resort
42,Bubaque,GW,80.89,overcast clouds,11.2833,-15.8333,"Casa Dora, Bubaque"


In [10]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247 entries, 8 to 692
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 247 non-null    object 
 1   Country              247 non-null    object 
 2   Max Temp             247 non-null    float64
 3   Current Description  247 non-null    object 
 4   Lat                  247 non-null    float64
 5   Lng                  247 non-null    float64
 6   Hotel Name           225 non-null    object 
dtypes: float64(3), object(4)
memory usage: 23.5+ KB


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [13]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Rikitea,PF,75.00,clear sky,-23.1203,-134.9692,People ThankYou
10,Vila Velha,BR,75.15,clear sky,-20.3297,-40.2925,Hotel Vitória Palace
17,Kapaa,US,84.52,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
25,Hilo,US,87.28,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
28,Akyab,MM,80.53,overcast clouds,20.1500,92.9000,Hotel Memory
33,Puerto Colombia,CO,84.29,broken clouds,10.9878,-74.9547,Hotel El Emigrante
36,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,Keleti Beach Resort
42,Bubaque,GW,80.89,overcast clouds,11.2833,-15.8333,"Casa Dora, Bubaque"
45,Lompoc,US,87.01,clear sky,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
51,San Ramon,US,88.57,clear sky,37.7799,-121.9780,San Ramon Marriott


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} &deg;F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, 
                                 # dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))